In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu copy


In [12]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [13]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [14]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [15]:
def user_function_co2(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['duration'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [16]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [17]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_co2(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(50)
test_Y = user_function_co2(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(270)
train_Y = user_function_co2(train_X)

[[14.         18.59805717 19.51134292 65.19823397  2.          2.61577094]
 [ 9.         23.20544458 13.2807693  43.81955176  2.          1.53559613]
 [11.         17.20213138 10.91735224 69.90123432  1.          2.93029956]
 [17.         16.73615338 22.76976615 67.15033779  1.          2.47049244]
 [ 6.         17.76049183 10.07338666 50.24647143  2.          1.56798354]
 [17.         14.16070016 26.31776149 53.18111819  2.          4.95396074]
 [16.         13.31180161  9.33740209 35.83310666  2.          3.51909411]
 [10.          8.45983218  5.60962995 50.73242108  2.          4.40854239]
 [ 6.         19.2177413  25.01191026 47.50831046  3.          4.08141935]
 [18.          8.10118311 23.36086682 56.69891274  2.          4.10337899]
 [ 4.         15.82786608  9.32350735 56.55912405  2.          3.49602501]
 [18.         13.58804079 26.71059637 60.57827566  1.          1.66663273]
 [ 5.         20.76794078 12.73488779 40.05228187  2.          3.40755296]
 [ 4.         20.48113541


Evaluating input: 36 of 50

 Retrying in 1 seconds

Evaluating input: 37 of 50

 Retrying in 1 seconds

Evaluating input: 38 of 50

 Retrying in 1 seconds

Evaluating input: 39 of 50

 Retrying in 1 seconds

Evaluating input: 40 of 50

 Retrying in 1 seconds

Evaluating input: 41 of 50

 Retrying in 1 seconds

Evaluating input: 42 of 50

 Retrying in 1 seconds

Evaluating input: 43 of 50

 Retrying in 1 seconds

Evaluating input: 44 of 50

 Retrying in 1 seconds

Evaluating input: 45 of 50

 Retrying in 1 seconds

Evaluating input: 46 of 50

 Retrying in 1 seconds

Evaluating input: 47 of 50

 Retrying in 1 seconds

Evaluating input: 48 of 50

 Retrying in 1 seconds

Evaluating input: 49 of 50

 Retrying in 1 seconds

Evaluating input: 50 of 50

 Retrying in 1 seconds
[[12.         17.27478322 24.8192092  69.49051878  1.          1.54002918]
 [12.         18.75206927 10.76793955 59.34246187  1.          2.03032969]
 [20.         16.82212342 24.75484277 66.23720877  1.          3.96478


Evaluating input: 131 of 270

 Retrying in 1 seconds

Evaluating input: 132 of 270

 Retrying in 1 seconds

Evaluating input: 133 of 270

 Retrying in 1 seconds

Evaluating input: 134 of 270

 Retrying in 1 seconds

Evaluating input: 135 of 270

 Retrying in 1 seconds

Evaluating input: 136 of 270

 Retrying in 1 seconds

Evaluating input: 137 of 270

 Retrying in 1 seconds

Evaluating input: 138 of 270

 Retrying in 1 seconds

Evaluating input: 139 of 270

 Retrying in 1 seconds

Evaluating input: 140 of 270

 Retrying in 1 seconds

Evaluating input: 141 of 270

 Retrying in 1 seconds

Evaluating input: 142 of 270

 Retrying in 1 seconds

Evaluating input: 143 of 270

 Retrying in 1 seconds

Evaluating input: 144 of 270

 Retrying in 1 seconds

Evaluating input: 145 of 270

 Retrying in 1 seconds

Evaluating input: 146 of 270

 Retrying in 1 seconds

Evaluating input: 147 of 270

 Retrying in 1 seconds

Evaluating input: 148 of 270

 Retrying in 1 seconds

Evaluating input: 149 of 27

### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [18]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = 96.71579658393532
Optimization restart 2/5, f = 114.06752556100139
Optimization restart 3/5, f = 96.7157963262488
Optimization restart 4/5, f = 114.06748678248019
Optimization restart 5/5, f = 96.71579698436986


GP_regression.,value,constraints,priors
rbf.variance,8200.388523850372,+ve,
rbf.lengthscale,32.716963025359554,+ve,
Gaussian_noise.variance,327.17140185899643,+ve,


#### Acquisition Function

In [19]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [20]:
optimizer = GradientAcquisitionOptimizer(parameter_space)


#### Experimental Design Loop

In [21]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [22]:
ed_loop.run_loop(user_function_co2, 250)

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 96.7157963262488
Optimization restart 2/5, f = 114.06747814534127
Optimization restart 3/5, f = 114.06747805235322
Optimization restart 4/5, f = 114.06750096369906
Optimization restart 5/5, f = 114.06759014672882
[[20.  25.  27.  30.   1.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 101.44492187667174
Optimization restart 2/5, f = 119.71798333802639
Optimization restart 3/5, f = 119.71797530310369
Optimization restart 4/5, f = 119.71797505536416
Optimization restart 5/5, f = 101.42605055209187
[[ 3. 25. 27. 70.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


...

Optimization restart 1/5, f = 1292.055408632071
Optimization restart 2/5, f = 1292.055404603697
Optimization restart 3/5, f = 1335.4106544680187
Optimization restart 4/5, f = 1292.0554046104662
Optimization restart 5/5, f = 1613.7446747887884
[[20.         15.92164496 27.         37.73493204  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1295.9346144420083
Optimization restart 2/5, f = 1295.9345655894003
Optimization restart 3/5, f = 1619.6384753382363
Optimization restart 4/5, f = 1295.9345656958749
Optimization restart 5/5, f = 1620.5498674213025
[[11.         13.77030657 17.40817803 70.          3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1299.8180518997706
Optimization restart 2/5, f = 1299.8178398380155
Optimization restart 3/5, f = 1342.8600811598967
Optimization restart 4/5, f = 1299.8178401126736
Optimization restart 5/5, f = 1625.4011112254025


### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [23]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = 96.71579658393532
Optimization restart 2/5, f = 114.0674779764203
Optimization restart 3/5, f = 96.71579632125928
Optimization restart 4/5, f = 114.06748073000765
Optimization restart 5/5, f = 114.06748713953534


#### Acquisition Function

In [24]:
integrated_variance_reduction = IntegratedVarianceReduction(
    model=emukit_model_ird, 
    space=parameter_space, 
    num_monte_carlo_points=100
)

#### Optimizer

In [25]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [26]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [27]:
ed_loop.run_loop(user_function_co2, 250)

Optimization restart 1/5, f = 96.71579632125928
Optimization restart 2/5, f = 114.06751261374158
Optimization restart 3/5, f = 96.71579647128004
Optimization restart 4/5, f = 114.06753259151645
Optimization restart 5/5, f = 96.71579632295025
[[ 3.         15.72072557 27.         70.          2.          4.40185406]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 102.61006240192418
Optimization restart 2/5, f = 109.42372341158051
Optimization restart 3/5, f = 121.51206465734865
Optimization restart 4/5, f = 102.61006239774805
Optimization restart 5/5, f = 121.51207951744735
[[20.         16.82457754 26.88787997 30.          1.          3.41430513]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 1/5, f = 107.09278064975663
Optimization restart 2/5, f = 127.37307917691751
Optimization restart 3/5, f = 114.11909302036776
Optimization restart 4/5, f = 114.13656132935535
Optimization restart 5/5, f = 127.37321592143776
[[20.         18.15251055  5.         68.16710035  3.          2.81024173]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


...

Optimization restart 1/5, f = 1277.5657085785401
Optimization restart 2/5, f = 1277.5656908035699
Optimization restart 3/5, f = 1277.5656922309915
Optimization restart 4/5, f = 1277.5656907286873
Optimization restart 5/5, f = 1277.5656907281066
[[14.         22.4327007   8.70110434 44.87079056  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 1/5, f = 1281.5337415658005
Optimization restart 2/5, f = 1281.5336130798123


 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 3/5, f = 1512.2247500146277
Optimization restart 4/5, f = 1281.5336130723251
Optimization restart 5/5, f = 1281.533611399917


### Random GP

Supervised learning

In [28]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=50)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/50, f = 1215.6460638557614
Optimization restart 2/50, f = 1215.646063931534
Optimization restart 3/50, f = 1609.0461359920894
Optimization restart 4/50, f = 1215.6460638575493
Optimization restart 5/50, f = 1215.646063855508
Optimization restart 6/50, f = 1609.046148844121
Optimization restart 7/50, f = 1215.6460638563105
Optimization restart 8/50, f = 1215.6460638582978
Optimization restart 9/50, f = 1609.0461291583383
Optimization restart 10/50, f = 1215.6460638555418
Optimization restart 11/50, f = 1215.6460638557176
Optimization restart 12/50, f = 1215.6460638567937
Optimization restart 13/50, f = 1215.6460638556123
Optimization restart 14/50, f = 1215.6460638563067
Optimization restart 15/50, f = 1215.646063855534
Optimization restart 16/50, f = 1215.6460638556994
Optimization restart 17/50, f = 1609.0461344337118
Optimization restart 18/50, f = 1215.6460638553885
Optimization restart 19/50, f = 1215.6460638555757
Optimization restart 20/50, f = 1215.6460638

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:108: RuntimeWarning:invalid value encountered in greater
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/paramz/transformations.py:113: RuntimeWarning:invalid value encountered in greater


Warning - optimization restart 28/50 failed
Optimization restart 29/50, f = 1215.6460638571657
Optimization restart 30/50, f = 1609.0461282250762
Optimization restart 31/50, f = 1215.6460638769731
Optimization restart 32/50, f = 1609.0461293161081
Optimization restart 33/50, f = 1215.646063861138
Optimization restart 34/50, f = 1215.6460638557078
Optimization restart 35/50, f = 1215.6460638599833
Optimization restart 36/50, f = 1215.646063856094
Optimization restart 37/50, f = 1215.6460639156535
Optimization restart 38/50, f = 1215.6460638562214
Optimization restart 39/50, f = 1215.6460638554386
Optimization restart 40/50, f = 1215.6460638554074
Optimization restart 41/50, f = 1215.6460638642334
Optimization restart 42/50, f = 1215.646063860853
Optimization restart 43/50, f = 1424.3050391083952
Optimization restart 44/50, f = 1215.646063855416
Optimization restart 45/50, f = 1215.6460638574986
Optimization restart 46/50, f = 1215.6460638573662
Optimization restart 47/50, f = 1215.64606

GP_regression.,value,constraints,priors
rbf.variance,149299.40286959006,+ve,
rbf.lengthscale,26.641767889590593,+ve,
Gaussian_noise.variance,226.41291471145917,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [29]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  19.681160404159662


In [30]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  15.017247025212955


In [31]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  19.354252524393146


In [32]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is -1.689% better than training on random points.'

In [33]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is 22.409% better than training on random points.'

### Save Models

In [34]:
with open(f'experimental_design/models/model_variance_20_init_points_250_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)

with open(f'experimental_design/models/integrated_variance_reduction_20_init_points_250_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design/models/270_random_points_supervised_GP_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)